# <center> Tâche 1 de la pipeline : transcription des audios à l'écrit</center>

A partir de nos données audios (générées dans le notebook *Dataset_Creation*), nous allons les transcrire à l'écrit en utilisant un modèle de *speech to text*. Le **défi** ici est de parvenir à transcrire les audios en conservant les erreurs afin de permettre à l'utilisateur de visualiser ses erreurs dans le texte, et par la suite le texte corrigé. Le problème est que la plupart des modèles de *speech to text* possèdent une couche de traitement de la langue leur permettant de corriger les fautes. Un **autre défi** auquel nous allons faire face est l'affinnage du modèle, pour tenter d'augmenter ses performances en particulier sur l'accent indien.